In [18]:
from typing import List
from pydantic import BaseModel, Field
import pandas as pd 
import openai
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_community.document_loaders import DataFrameLoader
from typing import List
from pydantic import BaseModel, Field



In [19]:
os.environ["OPENAI_API_TYPE"]       = ""
os.environ["OPENAI_API_BASE"]       = ""
os.environ["OPENAI_API_KEY"]        = ""
os.environ["OPENAI_API_VERSION"]    = ""
openai.api_type     = os.environ["OPENAI_API_TYPE"]
openai.api_base     = os.environ["OPENAI_API_BASE"]
openai.api_key      = os.environ["OPENAI_API_KEY"] 
openai.api_version  = os.environ["OPENAI_API_VERSION"]

In [20]:
model = AzureChatOpenAI(deployment_name='' , verbose=False , temperature=0)

In [455]:
from typing import Literal
class Pos_tag(BaseModel):
    """Tag the sentences with the list of aspects"""
    #aspects:list=all_aspects
    tone: Literal["positive", "negative"]
    tags: list = Field(description="sentiment tag of text 'pos','neg','neutral ")

In [28]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [29]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the  text   as instructed"),
    ("user", " ['whitenning','long lasting']  should be mapped to find sentiment{input}")
])

In [37]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [38]:
tagging_chain = prompt | model_with_functions

In [39]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [40]:
tagging_chain.invoke({"input": "I love langchain"})

{'sentiment': 'pos', 'language': 'en'}

In [82]:
from langchain_community.document_transformers.openai_functions import (
    create_metadata_tagger,
)

In [119]:
from pydantic import BaseModel, Field
import json
from typing import Literal

class Tags(BaseModel):
    tone: Literal["positive", "negative","Neutral"]



In [121]:
prompt = ChatPromptTemplate.from_template(
"""
  tag sentiments for this {input} with respect to the string
"""
)

In [122]:

document_transformer = create_metadata_tagger(metadata_schema=Tags, llm=model,prompt=prompt)


In [109]:
original_documents =loader.load()[1:5]
import json

In [110]:
enhanced_documents = document_transformer.transform_documents(original_documents)

print(
    *[d.page_content + "\n\n" + json.dumps(d.metadata) for d in enhanced_documents],
    sep="\n\n---------------\n\n"
)

As a diligent user of FrostBrite toothpaste, I have tried various types to cater to my oral needs. While I appreciate the effectiveness of the product, I did notice that the toothpaste tends to dry out quickly in the tube. This can be a bit frustrating as I have to waste some of the product or struggle to squeeze it out. Nonetheless, this is a minor issue compared to the overall benefits of the toothpaste.

{"input": "positive", "File Name": "paste-review10-1.txt"}

---------------

As a health-conscious woman, I was excited to try FrostBrite toothpaste with its range of natural ingredients and eco-friendly packaging. However, I was disappointed to find that the toothpaste dried out quickly in the tube, making it difficult to use. While I appreciate the emphasis on natural ingredients, I hope the brand can address this issue in the future to make their product more user-friendly. Overall, I still appreciate the effort put into creating a more sustainable toothpaste option.

{"input": "